# Import package

In [1]:
import os
import torch
import pandas as pd
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning import loggers
from pytorch_lightning.callbacks import ModelCheckpoint

In [2]:
%load_ext autoreload
%autoreload 2

# Setup environment

In [3]:
device = "cuda" if torch.cuda.is_available else "cpu"

def format_bytes(size):
    # 2**10 = 1024
    power = 2**10
    n = 0
    power_labels = {0 : '', 1: 'kilo', 2: 'mega', 3: 'giga', 4: 'tera'}
    while size > power:
        size /= power
        n += 1
    return "%.2f" % size, power_labels[n] + 'bytes'

torch.cuda.get_device_name(), format_bytes(torch.cuda.get_device_properties(device).total_memory)

('GeForce GTX 1080 Ti', ('10.92', 'gigabytes'))

# Run Teacher Inference

In [54]:
from trainer.distill.ban import BanStudentTrainer
model = BanStudentTrainer(1)
max_epoches = model.get_max_epoches()

>> Load from data frame


>> The transformation of student is
<function apply_policy at 0x7fce6ebaaee0>
>> Return trainloader
>> Return valloader


In [5]:
checkpoint_path = os.path.join(os.getcwd(), "checkpoint", model.model_name)
checkpoint_callback = ModelCheckpoint(
    filepath = checkpoint_path,
    save_top_k = 5,
    verbose = True,
    monitor = 'kappa_score',
    mode = 'max'
)

checkpoint_path

NameError: name 'model' is not defined

In [29]:
tb_logger = loggers.TensorBoardLogger('logs/', name = model.model_name)
trainer = pl.Trainer(checkpoint_callback = checkpoint_callback,
                    nb_sanity_val_steps = 0, 
                    max_epochs = max_epoches, 
                    gpus = -1, 
                    logger = tb_logger)

INFO:lightning:GPU available: True, used: True
INFO:lightning:CUDA_VISIBLE_DEVICES: [0]


# Learning finder

In [30]:
model.max_lr = 3e-2
model.configure_optimizers()

SGD (
Parameter Group 0
    base_momentum: 0.85
    dampening: 0
    initial_lr: 0.0012
    lr: 0.029999968800641534
    max_lr: 0.03
    max_momentum: 0.9
    min_lr: 1.1999999999999999e-05
    momentum: 0.8500000520197386
    nesterov: False
    weight_decay: 5e-05
)

# Start to train the model

In [31]:
trainer.fit(model)

INFO:lightning:
    | Name                             | Type              | Params
-------------------------------------------------------------------
0   | encoder                          | ResNetModel       | 66 M  
1   | encoder.encoder                  | Sequential        | 66 M  
2   | encoder.encoder.0                | Conv2d            | 9 K   
3   | encoder.encoder.1                | BatchNorm2d       | 128   
4   | encoder.encoder.2                | ReLU              | 0     
5   | encoder.encoder.3                | MaxPool2d         | 0     
6   | encoder.encoder.4                | Sequential        | 634 K 
7   | encoder.encoder.4.0              | Bottleneck        | 206 K 
8   | encoder.encoder.4.0.conv1        | Conv2d            | 8 K   
9   | encoder.encoder.4.0.bn1          | BatchNorm2d       | 256   
10  | encoder.encoder.4.0.conv2        | Conv2d            | 147 K 
11  | encoder.encoder.4.0.bn2          | BatchNorm2d       | 256   
12  | encoder.encoder.4.0.conv3 

>> Return trainloader
>> Return valloader


TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/kientiet/.local/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/kientiet/.local/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/kientiet/.local/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/kientiet/Documents/My Project/python/panda-kaggle/preprocessing/dataset/load_ban_dataset.py", line 38, in __getitem__
    img = transform(img)
  File "/home/kientiet/.local/lib/python3.8/site-packages/torchvision/transforms/transforms.py", line 61, in __call__
    img = t(img)
  File "/home/kientiet/Documents/My Project/python/panda-kaggle/preprocessing/augment/randaug/r